## Data Acquisition
This notebook was used to scrape a toy dataset from online vacancy postings. It will be used again to retrieve a larger final dataset. 
Mostly self explanatory but needs refactoring and some markdown to explain in order.

In [5]:
import requests
from bs4 import BeautifulSoup
import nltk
import matplotlib.pyplot as plt
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
import pandas as pd
from datetime import datetime
import time
import random
from langdetect import detect

In [2]:
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',}

In [3]:
def get_pages(search_term, page_count, country): #returns a list of search links given an input search term
    pages = []
    for page_no in range(page_count):
        page = 'https://'+country+'.indeed.com/jobs?q='+search_term+'&start='+str(page_no*10)
        pages.append(page)
    return pages

In [4]:
def get_links(search_link, country): #Takes in one search page and returns all job links on the page
    links = []
    page = requests.get(search_link, headers = headers)
    search_soup = BeautifulSoup(page.text)
    #print(search_soup)
    mydivs = search_soup.find_all("div", {"class": "mosaic mosaic-provider-jobcards"})
    for div in mydivs:
        classes = div.find_all("a")
        for c in classes:
            sub = c.get('data-jk')
            if isinstance(sub, str):
                link = 'https://'+country+'.indeed.com/viewjob?jk='+sub
            if link not in links:
                links.append(link)
    return links

In [7]:
countries = ['se','no','nl','de','be','es','it','ch','fr','uk','pt','fi','lu','at']
link_list = []
for country in countries:
    pages = get_pages("Data Scientist", 1, country)
    for page in pages:
        links = get_links(page, country)
        link_list.append(links)
print(link_list)

[['https://se.indeed.com/viewjob?jk=b0669075c820856d', 'https://se.indeed.com/viewjob?jk=3bac76bf62ea9208', 'https://se.indeed.com/viewjob?jk=dbfe7d7c087065d7', 'https://se.indeed.com/viewjob?jk=8721373c378ce53f', 'https://se.indeed.com/viewjob?jk=31bae7cd6472fcc2', 'https://se.indeed.com/viewjob?jk=841a8fdd1bb00f2f', 'https://se.indeed.com/viewjob?jk=88c25a48d2da8a7a', 'https://se.indeed.com/viewjob?jk=3ee63036a5c09fdb', 'https://se.indeed.com/viewjob?jk=703da12b0bd70a62', 'https://se.indeed.com/viewjob?jk=1b4b8ee005f06350', 'https://se.indeed.com/viewjob?jk=fef47e1d4e7e0f78', 'https://se.indeed.com/viewjob?jk=514330948751b10c', 'https://se.indeed.com/viewjob?jk=3bc7877aac35e665', 'https://se.indeed.com/viewjob?jk=9926cbcdcc05c9be', 'https://se.indeed.com/viewjob?jk=b0ea1fe650a0f37d'], ['https://no.indeed.com/viewjob?jk=fba1424ecfce6280', 'https://no.indeed.com/viewjob?jk=41a920a2afbb16a0', 'https://no.indeed.com/viewjob?jk=7cdc4821e14f8d5c', 'https://no.indeed.com/viewjob?jk=65bf98ba

In [8]:
def get_content(url): #Takes in a job link and returns structured content
    page = requests.get(url)
    search_soup = BeautifulSoup(page.text)
    full_text = search_soup.find("div",{"id": "jobDescriptionText"}).text
    if(detect(full_text) != 'en'):
        print(full_text[0:50]) #TESTING
        return 
    
    content = []
    title_location = search_soup.find("title").text
    title = title_location.split("-")[0]
    location = title_location.split("-")[1]
    country = url[8:10]
    timestamp = datetime.now()

    list_elements = []
    
    divs = search_soup.find_all("div",{"id": "jobDescriptionText"})
    for div in divs:
        uls = div.find_all("ul")
        for ul in uls:
            for li in ul.find_all('li'):
                list_elements.append(li.text)
    print(country, title)            #TESTING
    return [timestamp,url,title,location,country,full_text,list_elements]

In [47]:
def get_content_df(urls): #Runs get_content on a list of urls returning a pandas dataframe with content
    df = pd.DataFrame(columns = ["dt","url","title","location","country","full_text","list_elements"])
    for url in urls:
        content = get_content(url)
        if content: #because return call in get_content will return None, don't include in df
            df.loc[len(df)] = content
        time.sleep(10+random.uniform(-1,1)) #behave a bit like human
    return df

In [56]:
df_toy = get_content_df(link_list)

E.ON är övertygade om att du som är junior, nyfike
se Data Scientist 
se Data Scientist 
Är du en som går igång på att bygga briljanta anal
Vill du vara med och bidra till en mer god och säk
se Data Scientist 
se Data Scientist 
se Junior data scientist 
se Junior data scientist 
se Data Scientist, Search Insights 
se Data Scientist, Search Insights 
KOMPETENS
Data/IT
PUBLIKATIONSDATUM
4 Aug 2021
LAN
Funmed är en svensk MedTech-startup och Europas st
Micropower Group befinner sig i stark tillväxt och
Position Description:
Är du kreativ, kommunikativ 
se Data Scientist 
se Data Scientist 
se Data Scientist 
se Data Scientist 
se ESG Data Scientist 
se ESG Data Scientist 
se Data Scientist 
se Data Scientist 
no Data Scientist 
no Data Scientist 
no Data Scientist 
no Data Scientist 
no Data Scientist 
no Data Scientist 
En jobb i DI kan nesten ikke beskrives, den maÌŠ o
Siemens Mobility er verdens ledende leverandør av 
no Data Scientist 
no Data Scientist 
Vi er på jakt etter dyktige t

AttributeError: 'NoneType' object has no attribute 'text'

In [17]:
df_toy = pd.DataFrame(columns = ["dt","url","title","location","country","full_text","list_elements"])
for list in link_list[0:2]:
    df=df_toy.append([0,2,3])

In [61]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None): #no row limit
    print(df_toy)

                           dt  \
0  2021-08-08 21:01:03.303868   
1  2021-08-08 21:01:09.824428   
2  2021-08-08 21:01:18.030788   
3  2021-08-08 21:01:20.651507   
4  2021-08-08 21:01:27.135797   
5  2021-08-08 21:01:30.941859   
6  2021-08-08 21:01:33.159440   
7  2021-08-08 21:01:39.542159   
0  2021-08-09 08:12:45.318860   
1  2021-08-09 08:13:15.558980   
2  2021-08-09 08:13:47.583781   
3  2021-08-09 08:13:58.416752   
4  2021-08-09 08:14:31.794949   
5  2021-08-09 08:14:41.841642   
6  2021-08-09 08:15:03.340762   
7  2021-08-09 08:15:14.066627   
0  2021-08-09 08:15:34.813357   
1  2021-08-09 08:16:16.596947   
2  2021-08-09 08:16:27.980385   
3  2021-08-09 08:16:47.856741   
4  2021-08-09 08:17:09.942341   
5  2021-08-09 08:17:20.627842   
6  2021-08-09 08:17:30.731616   
7  2021-08-09 08:17:41.086215   
0  2021-08-09 08:30:43.101968   
1  2021-08-09 08:30:52.730759   
2  2021-08-09 08:31:14.301947   
3  2021-08-09 08:31:25.383901   
4  2021-08-09 08:32:17.141911   
5  2021-08

In [60]:
for list in link_list:
    df_toy = df_toy.append(get_content_df(list))
df_toy.to_csv("df_toy.csv")

de Junior Data Engineer / Data Scientist (m/w/d) 
de Field Application Scientist (Tissue Biomarker Reagents) covering DACH + Nor 
Die datasolut GmbH ist ein junge KI-Beratung aus K
de Jr Data Scientist 
de Data Scientist (f/m/d) 
Tools
Python
Keras
Spark
Scala
KNIME
Tableau
Gehal
Data Scientist (m/w/d) | Computer Vision | B2B | S
Zum Aufbau des innovativen Bereichs Business Intel
Wir sind eine Neugründung im Umfeld des Digitalhub
de Data Scientist/Machine Learning Engineer 
de Product Analyst / Data Scientist (m/f/d) 
de Data Scientist (m/w/d) 
Junior Data Scientist (m/w/d)
Plan D ist ein auf D
Wir von MONDAY.ROCKS haben ein digitales Team-Inst
de Data Scientist (f/m/d) wanted for Berlin! Ref.Nr. 2755 
De Lijn wil een oplossing zijn voor de mobiliteit 
be Data Scientist 
be AfinIT 
be Data Scientist for Supply Chain Sustainability (M/F/X) 
Cegeka is één van de leiders op het vlak van IT Co
Als data scientist / AI-engineer binnen Mediaan, m
be DATA SCIENTIST 
be Data Scientist 
be Junio